# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298339469821                   -0.85    4.9
  2   -8.300185834557       -2.73       -1.26    1.1
  3   -8.300447483807       -3.58       -1.90    4.0
  4   -8.300462204925       -4.83       -2.74    2.1
  5   -8.300464115228       -5.72       -2.99    5.4
  6   -8.300464359486       -6.61       -3.16    1.0
  7   -8.300464502583       -6.84       -3.30    1.1
  8   -8.300464578543       -7.12       -3.44    1.0
  9   -8.300464631113       -7.28       -3.66    1.0
 10   -8.300464633931       -8.55       -3.74    1.8
 11   -8.300464641498       -8.12       -3.95    1.0
 12   -8.300464643919       -8.62       -4.48    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67437586094                   -0.70    6.2
  2   -16.67861173169       -2.37       -1.14    1.6
  3   -16.67923154999       -3.21       -1.87    4.2
  4   -16.67928190625       -4.30       -2.77    3.2
  5   -16.67928614018       -5.37       -3.19    4.0
  6   -16.67928621782       -7.11       -3.48    4.0
  7   -16.67928622262       -8.32       -3.96    1.0
  8   -16.67928622422       -8.80       -4.59    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32526472157                   -0.56    6.9
  2   -33.33254681328       -2.14       -1.00    1.0
  3   -33.33412225077       -2.80       -1.73    4.9
  4   -33.33606041329       -2.71       -2.25    3.2
  5   -33.33690332374       -3.07       -2.47   16.0
  6   -33.33693317113       -4.53       -2.77    1.9
  7   -33.33694384184       -4.97       -3.66    2.0
  8   -33.33694391685       -7.12       -3.97    5.5
  9   -33.33694392662       -8.01       -4.56    1.1


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298361791363                   -0.85    5.2
  2   -8.300245871995       -2.72       -1.59    1.0
  3   -8.300430328988       -3.73       -2.43    4.2
  4   -8.300320558757   +   -3.96       -2.18   12.0
  5   -8.300464486341       -3.84       -3.68    3.2
  6   -8.300464577195       -7.04       -3.89    3.4
  7   -8.300464641486       -7.19       -4.59    6.5


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.30656105434                   -0.56    6.8
  2   -33.29269334774   +   -1.86       -1.25    1.9
  3   +4.807347176251   +    1.58       -0.28    8.1
  4   -33.32735057869        1.58       -1.72    7.0
  5   -33.20618576371   +   -0.92       -1.25    4.2
  6   -33.09392184490   +   -0.95       -1.37    4.2
  7   -33.25353719962       -0.80       -1.60    4.8
  8   -33.33668146629       -1.08       -2.31    3.5
  9   -33.33681728985       -3.87       -2.74    2.9
 10   -33.33688048430       -4.20       -2.88    2.8
 11   -33.33691846825       -4.42       -2.97    1.9
 12   -33.33692045603       -5.70       -3.20    2.0
 13   -33.33694293376       -4.65       -3.57    3.4
 14   -33.33694383567       -6.04       -4.15    3.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.